# Roteamento

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model='gpt-4o-mini')

prompt = ChatPromptTemplate.from_template("""Você é um professor de matemática de ensino fundamental
capaz de dar respostas muito detalhadas e didáticas. Responda a seguinte pergunta de um aluno:
Pergunta: {pergunta}""")
chain_matematica = prompt | model

prompt = ChatPromptTemplate.from_template("""Você é um professor de física de ensino fundamental
capaz de dar respostas muito detalhadas e didáticas. Responda a seguinte pergunta de um aluno:
Pergunta: {pergunta}""")
chain_fisica = prompt | model

prompt = ChatPromptTemplate.from_template("""Você é um professor de história de ensino fundamental
capaz de dar respostas muito detalhadas e didáticas. Responda a seguinte pergunta de um aluno:
Pergunta: {pergunta}""")
chain_historia = prompt | model

prompt = ChatPromptTemplate.from_template("""{pergunta}""")
chain_generica = prompt | model

In [ ]:
from pydantic import BaseModel, Field

prompt = ChatPromptTemplate.from_template("Você deve categorizar a seguinte pergunta: {pergunta}")

class Categorizador(BaseModel):
    """Categoriza as perguntas de alunos do ensino fundamental"""
    area_conhecimento: str = Field(description="A área de conhecimento da pergunta feita pelo aluno. \
    Deve ser 'física', 'matemática' ou 'história'. Caso não se encaixe em nenhuma delas, retorne 'outra'")

model_estruturado = prompt | model.with_structured_output(Categorizador)
model_estruturado.invoke({"pergunta": "Quando foi a independência dos Estados Unidos?"})

## Criando estrutura de roteamento

O RunnablePassthrough é um tipo de Runnable que simplesmente passa a entrada recebida como saída, podendo também adicionar chaves adicionais ao resultado. Ele se comporta quase como uma função de identidade, mas é útil em cenários onde você deseja incluir lógica adicional ou manipular a entrada de alguma forma antes de passar adiante.

In [ ]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough().assign(categoria=model_estruturado)
chain.invoke({'pergunta': 'Quando foi a independência dos Estados Unidos?'})

In [ ]:
def route(input):
    if input['categoria'].area_conhecimento == 'matemática':
        return chain_matematica
    if input['categoria'].area_conhecimento == 'física':
        return chain_fisica
    if input['categoria'].area_conhecimento == 'história':
        return chain_historia
    return chain_generica

In [ ]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough().assign(categoria=model_estruturado) | route
chain.invoke({'pergunta': 'Quando foi a independência dos Estados Unidos?'})